In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from copy import deepcopy
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

def modelfit(alg, train, test, predictors, target, \
             useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    """From Aarshay Jain's post Complete Guide to Parameter Tunning 
    in XGBoost
    """
    # make a copy
    dtrain = deepcopy(train)
    dtest  = deepcopy(test)
    
    # Cross validation
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, \
                              label=dtrain[target].values, \
                              missing=float('nan'))
        xgtest = xgb.DMatrix(dtest[predictors].values, \
                             label=dtest[target].values, \
                             missing=float('nan'))
        cvresult = xgb.cv(xgb_param, xgtrain, \
                          num_boost_round=alg.get_params()['n_estimators'], \
                          nfold=cv_folds, \
                          metrics=['auc'], \
                          early_stopping_rounds=early_stopping_rounds, \
                          show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='error')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob    = alg.predict_proba(dtrain[predictors])[:,1]
    dtrain['Prediction'] = pd.Series(dtrain_predictions, index=dtrain.index)
    dtrain['predprob']   = pd.Series(dtrain_predprob, index=dtrain.index)
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(
        dtrain[target].values, \
        dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(
        dtrain[target], \
        dtrain_predprob)
    
    #Predict on testing data:
    dtest_predictions = alg.predict(dtest[predictors])
    dtest_predprob    = alg.predict_proba(dtest[predictors])[:,1]
    dtest['Prediction'] = pd.Series(dtest_predictions, index=dtest.index)
    dtest['predprob']   = pd.Series(dtest_predprob, index=dtest.index)
    print "Accuracy : %.4g" % metrics.accuracy_score(
        dtest[target].values, \
        dtest_predictions)
    print 'AUC Score (Test): %f' % metrics.roc_auc_score(
        dtest[target], \
        dtest_predprob)
                
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

    return dtrain, dtest

In [11]:
target = 'earn_over_4k_euros_per_year'
index  = 'Id'
predictors = [x for x in train.columns if x not in [target,index]]

dftrain = pd.read_csv('../data/train.csv')
dftest = pd.read_csv('../data/test.csv')

In [12]:
train, test = train_test_split(dftrain, test_size=0.2)
train_semi = pd.concat([train, dftest])

In [13]:
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(train_semi[predictors], train_semi[target])
train_semi[target] = pd.Series(label_prop_model.tranduction_, \
                               index=train_semi.index)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').